In [45]:
using LinearAlgebra

In [46]:
# Функция для вычисления степени диагонального преобладания
function diagonal_dominance_degree(A)
    n = size(A, 1)
    dominance = -Inf  # Минимальное значение по всем строкам
    for i in 1:n
        diag_elem = abs(A[i, i])
        row_sum = sum(abs.(A[i, :])) - diag_elem
        dominance = max(dominance, diag_elem - row_sum)
    end
    return dominance
end

# Функция для генерации матрицы с заданной степенью диагонального преобладания
function generate_matrix_with_dominance(n, target_dominance)
    A = randn(n, n)
    
    # Делаем матрицу с заданным диагональным преобладанием
    for i in 1:n
        row_sum = sum(abs.(A[i, :])) - abs(A[i, i])
        if target_dominance >= 0
            A[i, i] = sign(A[i, i]) * (row_sum + target_dominance + abs(randn() * 0.1))
        else
            A[i, i] = sign(A[i, i]) * (row_sum + target_dominance - abs(randn() * 0.1))
        end
    end
    
    return A
end

generate_matrix_with_dominance (generic function with 1 method)

In [47]:
function jacobi_method_complete(A::Matrix{Float64}, f::Vector{Float64}, 
                               x0::Vector{Float64}, tol::Float64=1e-6, max_iter::Int=1000)
    n = length(f)
    
    # Построение матрицы P: P = -D⁻¹(A - D)
    D = Diagonal(diag(A))
    D_inv = inv(D)
    P = -D_inv * (A - D)
    g = D_inv * f         
    
    x = copy(x0)
    
    for k in 1:max_iter
        # по формуле: X⁽ᵏ⁾ = PX⁽ᵏ⁻¹⁾ + g
        x_k = P * x + g 
        
        # Критерий остановки
        if norm(x_k - x) < tol
            return x_k, k
        end
        
        x = x_k
    end
    
    error("Метод Якоби не сошелся за $max_iter итераций")
end

jacobi_method_complete (generic function with 3 methods)

In [48]:
function seidel_method_complete(A::Matrix{Float64}, f::Vector{Float64}, 
                              x0::Vector{Float64}, tol::Float64=1e-6, max_iter::Int=1000)
    n = length(f)
    
    # Разложение матрицы A = D + Aminus + Aplus
    D = Diagonal(diag(A))                    # Диагональная часть
    Aminus = tril(A, -1)                         # Строго нижняя треугольная часть
    Aplus = triu(A, 1)                          # Строго верхняя треугольная часть
    
    # Матрица P для метода Зейделя: P = -(D + Aminus)⁻¹Aplus
    P = -inv(D + Aminus) * Aplus                     
    g = inv(D + Aminus) * f                      
    
    x = copy(x0)
    
    for k in 1:max_iter
        # X⁽ᵏ⁾ = PX⁽ᵏ⁻¹⁾ + g
        x_k = P * x + g                  
        
        # Критерий остановки
        if norm(x_k - x) < tol
            return x_k, k
        end
        
        x = x_k
    end
    
    error("Метод Зейделя не сошелся за $max_iter итераций")
end

seidel_method_complete (generic function with 3 methods)

In [49]:
# Тестирование методов Якоби и Зейделя
function test_methods()
    println("ТЕСТИРОВАНИЕ ИТЕРАЦИОННЫХ МЕТОДОВ")
    println("="^50)
    
    # Система с диагональным преобладанием
    A = generate_matrix_with_dominance(3, 2)
    
    f = [15.0, 10.0, 10.0]
    x0 = zeros(3)
    
    println("Система AX = f:")
    println("A =")
    println(A)
    println("f = ", f)
    println("Начальное приближение x₀ = ", x0)
    println()
    
    # Точное решение для проверки
    x_exact = A \ f
    println("Точное решение: ", round.(x_exact, digits=6))
    println()
    
    # Метод Якоби с замером времени
    println("МЕТОД ЯКОБИ:")
    println("-"^40)
    time_j = @elapsed x_j, iter_j = jacobi_method_complete(A, f, x0)
    println("Полученное решение: ", round.(x_j, digits=6))
    println("Погрешность: ", norm(x_j - x_exact))
    println("Количество итераций: ", iter_j)
    println("Время выполнения: ", round(time_j, digits=6), " секунд")
    println()
    
    # Метод Зейделя с замером времени
    println("МЕТОД ЗЕЙДЕЛЯ:")
    println("-"^40)
    time_s = @elapsed x_s, iter_s = seidel_method_complete(A, f, x0)
    println("Полученное решение: ", round.(x_s, digits=6))
    println("Погрешность: ", norm(x_s - x_exact))
    println("Количество итераций: ", iter_s)
    println("Время выполнения: ", round(time_s, digits=6), " секунд")
    println()
    
    # Сравнение эффективности методов
    println("СРАВНЕНИЕ:")
    println("-"^40)
    println("Метод Якоби: $iter_j итераций, ", round(time_j, digits=6), " секунд")
    println("Метод Зейделя: $iter_s итераций, ", round(time_s, digits=6), " секунд")
    
    if iter_s < iter_j
        println("Метод Зейделя сходится быстрее по количеству итераций")
    end
end

test_methods()

ТЕСТИРОВАНИЕ ИТЕРАЦИОННЫХ МЕТОДОВ
Система AX = f:
A =
[2.666378855054332 0.42541675362351555 0.19899931893568856; -0.10500294879167169 -2.3494027321930813 0.17139226794107965; 0.29082987534847565 -1.170324656317088 -3.630933013643643]
f = [15.0, 10.0, 10.0]
Начальное приближение x₀ = [0.0, 0.0, 0.0]

Точное решение: [6.415857, -4.598448, -0.758042]

МЕТОД ЯКОБИ:
----------------------------------------
Полученное решение: [6.415857, -4.598448, -0.758042]
Погрешность: 1.9608198931892438e-7
Количество итераций: 10
Время выполнения: 1.5e-5 секунд

МЕТОД ЗЕЙДЕЛЯ:
----------------------------------------
Полученное решение: [6.415857, -4.598448, -0.758042]
Погрешность: 2.556011662289701e-8
Количество итераций: 6
Время выполнения: 5.1e-5 секунд

СРАВНЕНИЕ:
----------------------------------------
Метод Якоби: 10 итераций, 1.5e-5 секунд
Метод Зейделя: 6 итераций, 5.1e-5 секунд
Метод Зейделя сходится быстрее по количеству итераций
